In [1]:
import os
import pandas as pd
import xarray as xr
import numpy as np
import geopandas as gpd
import hydromt
from hydromt import DataCatalog
from hydromt_sfincs import SfincsModel, utils

In [2]:
'''

DESCRIPTION:
    This script processes SFINCS model output to attribute peak water levels to the different flood processes or drivers.
    Pieces of this code can be reused for a similar analysis.

AUTHOR: Lauren Grimley
CONTACT: lauren.grimley@unc.edu
Updated: 8/6/24

Input:
    SFINCS model
    Data array of downscaled peak flood depths

'''

In [3]:
def classify_zsmax_by_driver(da, compound_key, runoff_key, coastal_key, name_out, hmin):
    '''
    Input
        da = xr.datarray with the max water levels/flood depths for SFINCS runs where the index
             is the model name to be queried using the compound, runoff, and coastal keys.
     
        hmin = minimum difference in water level between the compound and max individual
    
    Output:
        da_classified = data array where the peak water level is attributed to a single flood processes using the hmin where
                        (0 = none, 1 = coastal, 2 = coastal compounded, 3=runoff, 4=runoff compounded)

        fld_area_by_driver = dataframe that is the total number of cells attributed to each flood process. 
                             This is used to calculate flood extent.

        da_compound = data array mask of the compound peak flood extent

        da_diff = data array of the compound scenario peak water levels minus the maximum individual
    
    '''
    # Calculate the max water level at each cell across the coastal and runoff drivers
    da_single_max = da.sel(run=[runoff_key, coastal_key]).max('run')
    
    # Calculate the difference between the max water level of the compound and the max of the individual drivers
    da_diff = (da.sel(run=compound_key) - da_single_max).compute()
    da_diff.name = 'diff in waterlevel compound minus max. single driver'
    da_diff.attrs.update(unit='m')

    # Create masks based on the driver that caused the max water level given a depth threshold hmin
    compound_mask = da_diff > hmin
    coastal_mask = da.sel(run=coastal_key).fillna(0) > da.sel(run=[runoff_key]).fillna(0).max('run')
    runoff_mask = da.sel(run=runoff_key).fillna(0) > da.sel(run=[coastal_key]).fillna(0).max('run')
    assert ~np.logical_and(runoff_mask, coastal_mask).any()
    da_classified = (xr.where(coastal_mask, x=compound_mask + 1, y=0)
                     + xr.where(runoff_mask, x=compound_mask + 3, y=0)).compute()
    da_classified.name = name_out

    # Calculate the number of cells that are attributed to the different drivers
    unique_codes, fld_area_by_driver = np.unique(da_classified.data, return_counts=True)

    # Return binary mask of compound peak flood extent
    da_compound = xr.where(compound_mask, x=1, y=0)
    da_compound.name = name_out

    return da_classified, fld_area_by_driver, da_compound, da_diff


def get_depth_stats(da_depth, var, thresholds):
    df_depth = da_depth.to_dataframe()
    df_depth.dropna(axis=0, inplace=True)
    df_depth = pd.DataFrame(df_depth[var])

    df_ss = pd.DataFrame()
    for threshold in thresholds:
        df_depth_sub = df_depth[df_depth[var] > threshold].astype(float).round(3)
        depth_stats = df_depth_sub.describe(percentiles=[0.05, 0.5, 0.95])
        depth_stats.columns = [f'depth_stats_{threshold}m']
        df_ss = pd.concat([df_ss, depth_stats], axis=1, ignore_index=False)

    return df_ss

In [4]:
# Filepath to data catalog yml; read in model
cat_dir = r'Z:\users\lelise\data'
yml_base_CONUS = os.path.join(cat_dir, 'data_catalog_BASE_CONUS.yml')
yml_base_Carolinas = os.path.join(cat_dir, 'data_catalog_BASE_Carolinas.yml')
yml_sfincs_Carolinas = os.path.join(cat_dir, 'data_catalog_SFINCS_Carolinas.yml')
os.chdir(r'Z:\users\lelise\projects\Carolinas_SFINCS\Chapter1_FlorenceValidation\sfincs_models\mod_v4_flor')
model_root = r'ENC_200m_sbg5m_avgN_adv1_eff75'
mod = SfincsModel(model_root, mode='r', data_libs=[yml_base_CONUS, yml_base_Carolinas, yml_sfincs_Carolinas])

In [5]:
# Load peak flood maps and print run keys
# Notes: the netcdf of peak water levels was created using the downscale_floodmaps.py script
res = 200
da = xr.open_dataarray(os.path.join(os.getcwd(), 'floodmaps',f'{res}m','floodmaps.nc'))
print(da.run.values)

['ENC_200m_sbg5m_avgN_adv1_eff75' 'ENC_200m_sbg5m_avgN_adv1_eff75_coastal'
 'ENC_200m_sbg5m_avgN_adv1_eff75_runoff'
 'ENC_200m_sbg5m_avgN_adv1_eff75_discharge'
 'ENC_200m_sbg5m_avgN_adv1_eff75_rainfall'
 'ENC_200m_sbg5m_avgN_adv1_eff75_stormTide'
 'ENC_200m_sbg5m_avgN_adv1_eff75_wind']


In [6]:
# Rename run IDs 
rename = ['compound','coastal','runoff','discharge','rainfall','stormTide','wind']
da['run'] = xr.IndexVariable('run', rename) 
print(da.run.values)

['compound' 'coastal' 'runoff' 'discharge' 'rainfall' 'stormTide' 'wind']


In [7]:
# Load in HUC6 watershed boundary
huc_boundary = gpd.read_file(r'Z:\users\lelise\data\geospatial\hydrography\nhd\NHD_H_North_Carolina_State_Shape\Shape\WBDHU6.shp')
huc_boundary.to_crs(32617, inplace=True)
huc_boundary = huc_boundary[["HUC6", "Name", "geometry"]]

In [8]:
# Create output directory to write files to
out_dir = os.path.join(os.getcwd(), 'process_attribution',f'{res}m')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
os.chdir(out_dir)

In [9]:
# Give the output filenames a unique ID
outfile_ext = 'all'

# # Comment the code out below if you want to get the full flood extent
# # Create a raster mask of the large water bodies and gets overland flooding
# coastal_wb = mod.data_catalog.get_geodataframe('carolinas_coastal_wb')
# coastal_wb.to_crs(epsg=32617, inplace=True)
# coastal_wb = coastal_wb.clip(mod.region)
# coastal_wb["water"] = 0
# coastal_wb = da.raster.rasterize(coastal_wb, "water", nodata=1, all_touched=False)

# # Mask the data that is located in large water bodies/estuaries
# da = da.where(coastal_wb == 1)

In [10]:
''' Attribute total flood extent to runoff, coastal, and compound processes '''
out = classify_zsmax_by_driver(da=da,
                               compound_key='compound',
                               runoff_key='runoff',
                               coastal_key='coastal',
                               name_out=out_dir,
                               hmin=0.05
                               )
da_c, fld_cells_by_driver, da_compound, da_diff = out

# Write to files
da_c.to_netcdf(f'flor_peakWL_attributed_{outfile_ext}.nc')
da_compound.to_netcdf(f'flor_peakWL_compound_extent_{outfile_ext}.nc')
da_diff.to_netcdf(f'flor_peakWL_compound_minus_maxIndiv_{outfile_ext}.nc')
da_diff.raster.to_raster(f'flor_peakWL_compound_minus_maxIndiv_{outfile_ext}.tif', nodata=np.nan)

In [11]:
# Calculate flood area stats (sq.km)
fld_area = fld_cells_by_driver.copy()
res = da_diff.raster.res[0]  # meters
fld_area = fld_area * (res * res) / (1000 ** 2)  # square km
fld_area = pd.DataFrame(fld_area.T)
fld_area.index = ['None', 'Coastal', 'Coastal Compound', 'Runoff', 'Runoff Compound']
fld_area.to_csv(f'flor_peakWL_attributed_area_{outfile_ext}.csv')
print(fld_area)

                          0
None              153190.96
Coastal             7658.28
Coastal Compound    3198.96
Runoff             11333.92
Runoff Compound      777.88


In [12]:
# Calculate the difference in water level stats
var = 'diff in waterlevel compound minus max. single driver'
depth_stats = get_depth_stats(da_depth=da_diff, var=var, thresholds=[-9999.0, 0.0, 0.05])
depth_stats.to_csv(f'flor_peakWL_compound_minus_maxIndiv_stats_{outfile_ext}.csv')
print(depth_stats)

       depth_stats_-9999.0m  depth_stats_0.0m  depth_stats_0.05m
count         573024.000000     470083.000000       99443.000000
mean               0.021841          0.028908           0.098846
std                0.046988          0.048451           0.065905
min               -0.399000          0.000000           0.050000
5%                -0.010000          0.000000           0.053000
50%                0.005000          0.008000           0.081000
95%                0.094000          0.102000           0.239000
max                0.644000          0.644000           0.644000


In [13]:
''' Get flood area/depth information by HUC6 basin '''
mdf = pd.DataFrame()
for basin in ['Pamlico', 'Neuse', 'Cape Fear', 'Onslow Bay', 'Lower Pee Dee']:
    sub = huc_boundary[huc_boundary['Name'] == basin]
    sub["basin"] = 1
    b = da.raster.rasterize(sub, "basin", nodata=-9999.0, all_touched=False)
    da2_c = da_c.where(b == 1)
    da2_diff = da_diff.where(b==1)

    unique_codes, fld_cells_by_driver = np.unique(da2_c.data, return_counts=True)
    fld_area = fld_cells_by_driver.copy()

    fld_area = fld_area * (res * res) / (1000 ** 2)  # square km
    fld_area = pd.DataFrame(fld_area)
    fld_area.index = ['None', 'Coastal', 'Coastal Compound', 'Runoff', 'Runoff Compound', 'Compound']
    fld_area.columns = [basin]
    mdf = pd.concat([mdf, fld_area], axis=1, ignore_index=False)

mdf.to_csv(f'flor_peakWL_attributed_area_by_HUC.csv')
print(mdf)

                    Pamlico      Neuse  Cape Fear  Onslow Bay  Lower Pee Dee
None               10586.08   12249.92   20557.28     2055.68       23937.32
Coastal             3970.20     581.24     137.20     1666.28         960.96
Coastal Compound    1537.20    1224.48      86.52      280.44          70.12
Runoff               474.12    1461.04    2979.48      429.92        5988.48
Runoff Compound       94.52     208.44      93.88      237.60         143.44
Compound          159497.88  160434.88  152305.64   171490.08      145059.68


In [14]:
''' Part 3 - Attribute OVERLAND flood extent to flood drivers (e.g., forcings) '''

def compute_waterlevel_difference(da, scen_base, scen_keys=None):
    # Computer the difference in water level for compound compared to maximum single driver
    da_single_max = da.sel(run=scen_keys).max('run')
    da1 = (da.sel(run=scen_base) - da_single_max).compute()
    da1.name = 'diff. in waterlevel\ncompound - max. single driver'
    da1.attrs.update(unit='m')
    return da1

In [15]:
da1 = compute_waterlevel_difference(da=da,
                                    scen_base='coastal',
                                    scen_keys=['stormTide','wind', 'discharge', 'rainfall']
                                    )

In [16]:
dh = 0.05  # minimum difference in water level between compound scenario and max individual drivers
compound_mask = da1 > dh  # mask of the cells where compound flooding occured
# Now create masks for the individual drivers
surge_mask = da.sel(run='stormTide').fillna(0) > da.sel(
    run=['discharge', 'rainfall', 'wind']).fillna(0).max('run')
coastal_mask = da.sel(run='wind').fillna(0) > da.sel(
    run=['discharge', 'rainfall', 'stormTide']).fillna(0).max('run')
discharge_mask = da.sel(run='discharge').fillna(0) > da.sel(
    run=['wind', 'rainfall', 'stormTide']).fillna(0).max('run')
precip_mask = da.sel(run='rainfall').fillna(0) > da.sel(
    run=['wind', 'discharge', 'stormTide']).fillna(0).max('run')
# precip_mask = np.logical_and(precip_mask, da1 >= 0)

assert ~np.logical_and(precip_mask, surge_mask).any() and ~np.logical_and(precip_mask,
                                                                          coastal_mask).any() and ~np.logical_and(
    precip_mask, discharge_mask).any()

assert ~np.logical_and(discharge_mask, surge_mask).any() and ~np.logical_and(discharge_mask,
                                                                             coastal_mask).any()
assert ~np.logical_and(surge_mask, coastal_mask).any()

da_c = (
        + xr.where(surge_mask, x=compound_mask + 1, y=0)
        + xr.where(coastal_mask, x=compound_mask + 3, y=0)
        + xr.where(discharge_mask, x=compound_mask + 5, y=0)
        + xr.where(precip_mask, x=compound_mask + 7, y=0)
).compute()
da_c.name = None

# Output the data array with driver attribution.
# 1 = surge, 2 = surge compounded, 3 = wind, 4 = wind compounded
# 5 = discharge, 6 = discharge compounded, 7 = rainfall, 8 = rainfall compounded
da_c.to_netcdf(f'flor_peakWL_attributed_toDrivers_{outfile_ext}.nc')